In [1]:
import pandas as pd
import random as rnd 
import mysql.connector
from mysql.connector import Error
import plotly
import json
import csv
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pymysql
import plotly.express as px
import plotly.graph_objects as go
from sqlalchemy import create_engine
import networkx as nx
from networkx_viewer import Viewer
from scipy.spatial.distance import pdist, squareform
import igraph
import sklearn
from sklearn.cluster import KMeans
from functools import reduce

## Get data from SQL servers

In [2]:
connection = mysql.connector.connect(host='localhost',
                                         database='employment',
                                         user='root',
                                         password='1234')

In [3]:
def get_data(connection, descriptor_domain,filt = False,val = None):
    '''
    Return df in the SOCs-decriptor format of a specific domain 
    descriptor_domeain:
        work context
        knowledge
        skills
        gwas
        job_zones, filter
        Abilities, filter
    
    '''
    df= pd.DataFrame(data=None)
    if descriptor_domain == 'work_context':
        df= pd.read_sql('''
            SELECT title, element_name, data_value FROM employment.work_context
            LEFT JOIN employment.content_model_reference USING (element_id) 
            LEFT JOIN employment.occupation_data USING (onetsoc_code)
            WHERE scale_id LIKE 'CX';''', con = connection)
        df = df.reset_index().pivot(index = 'title', columns = 'element_name', values = 'data_value')
    elif descriptor_domain == 'job_zones':
        if filt:
            df= pd.read_sql('''
                SELECT title,job_zone FROM employment.job_zones
                LEFT JOIN employment.occupation_data USING (onetsoc_code); ''', con = connection)
            df = df[df['job_zone'].isin(val)]
        else:
            df= pd.read_sql('''
                SELECT title,job_zone FROM employment.job_zones
                LEFT JOIN employment.occupation_data USING (onetsoc_code);''', con = connection)
    elif descriptor_domain == 'abilities':
        if filt:
            df= pd.read_sql('''
            SELECT title, element_name, data_value FROM employment.''' + descriptor_domain + '''
            LEFT JOIN employment.content_model_reference USING (element_id) 
            LEFT JOIN employment.occupation_data USING (onetsoc_code)
            WHERE element_id LIKE\''''+ val +'''%\'; ''', con = connection)
            
        else:
            df= pd.read_sql('''
            SELECT title, element_name,element_id, data_value FROM employment.''' + descriptor_domain + '''
            LEFT JOIN employment.content_model_reference USING (element_id) 
            LEFT JOIN employment.occupation_data USING (onetsoc_code);''', con = connection)
        df = df.groupby(by=['title','element_name'])['data_value'].mean()
        df = df.reset_index().pivot(index = 'title', columns = 'element_name', values = 'data_value')
    elif descriptor_domain == 'work_values':
        df= pd.read_sql('''
            SELECT title, element_name, data_value FROM employment.work_values
            LEFT JOIN employment.content_model_reference USING (element_id) 
            LEFT JOIN employment.occupation_data USING (onetsoc_code)
            WHERE scale_id LIKE 'EX';''', con = connection)
        df = df.reset_index().pivot(index = 'title', columns = 'element_name', values = 'data_value')
    elif descriptor_domain == 'interests':
        df= pd.read_sql('''
            SELECT title, element_name, data_value FROM employment.interests
            LEFT JOIN employment.content_model_reference USING (element_id) 
            LEFT JOIN employment.occupation_data USING (onetsoc_code)
            WHERE scale_id LIKE 'OI';''', con = connection)
        df = df.reset_index().pivot(index = 'title', columns = 'element_name', values = 'data_value')
    elif descriptor_domain == 'work_styles':
        df= pd.read_sql('''
            SELECT title, element_name, data_value FROM employment.work_styles
            LEFT JOIN employment.content_model_reference USING (element_id) 
            LEFT JOIN employment.occupation_data USING (onetsoc_code)
            WHERE scale_id LIKE 'IM';''', con = connection)
        df = df.reset_index().pivot(index = 'title', columns = 'element_name', values = 'data_value')
    elif descriptor_domain == 'name':
        df= pd.read_sql('''
            SELECT * FROM employment.occupation_data;''', con = connection)
    else:
        df= pd.read_sql('''
            SELECT title, element_name, data_value FROM employment.''' + descriptor_domain + '''
            LEFT JOIN employment.content_model_reference USING (element_id) 
            LEFT JOIN employment.occupation_data USING (onetsoc_code);''', con = connection)
        df = df.groupby(by=['title','element_name'])['data_value'].mean()
        df = df.reset_index().pivot(index = 'title', columns = 'element_name', values = 'data_value')
    return df

## Calculating euclidean distance 

In [4]:
# def distance_table(df, jz=False, Z = False):
#     '''
#     @param df: dataframe of 
    
#     '''
# #     squareform(pdist(df.iloc[:, 1:])), columns=df.title.unique(), index=df.title.unique()
#     dist = []
#     name_dict = {}
#     if jz:
#         name_dict = df['title'].sort_values().reset_index()['title'].to_dict()
#         dist = pdist(df[df.columns[1:]],'euclidean') #Euclidean distance for 1XN
#     else:
#         for i in range(len(df.index)):
#             name_dict[i] = df.index[i]
#         dist = pdist(df,'euclidean') #Euclidean distance for 1XN
#     df_dist = pd.DataFrame(squareform(dist)) #Euclidean distance for NxN
# #     return df_dist
#     columns = list(df_dist.index) 
#     for i in range(len(df.index)): #convert 0 values to NaN
#         df_dist.loc[i,i] = np.NaN
#     mean = df_dist.mean().mean()
#     std = df_dist.stack().std() 
    
#     if Z:
#         for col in columns: #Z-score calculated 
#             df_dist[col] = (df_dist[col] - mean)/std
# #     return df_dist
#     return df_dist.rename(columns = name_dict, index = name_dict)

In [5]:
df_skills = get_data(connection,'skills')

df_ab = get_data(connection,'abilities')

df_wc = get_data(connection,'work_context')

df_knowledge = get_data(connection,'knowledge')

df_gwas = get_data(connection,'work_activities')

df_jz = get_data(connection,'job_zones')
df_jz = df_jz[df_jz['title'] != 'Legislators']
# df_skills[:10]

In [6]:
df_skills

element_name,Active Learning,Active Listening,Complex Problem Solving,Coordination,Critical Thinking,Equipment Maintenance,Equipment Selection,Installation,Instructing,Judgment and Decision Making,...,Science,Service Orientation,Social Perceptiveness,Speaking,Systems Analysis,Systems Evaluation,Technology Design,Time Management,Troubleshooting,Writing
title,,,,,,,,,,,,,,,,,,,,,
Accountants,3.185,3.940,3.185,3.000,3.815,0.50,0.500,0.500,2.815,3.500,...,1.750,3.000,3.000,3.750,3.435,2.940,1.185,3.060,0.815,3.435
Actors,2.620,3.685,2.750,2.940,3.000,0.50,0.500,0.500,2.685,2.940,...,1.000,2.185,3.750,3.940,1.810,1.810,0.815,3.000,0.500,3.130
Actuaries,3.750,4.060,4.310,3.000,4.500,0.50,0.500,0.500,3.250,4.500,...,2.060,3.000,2.880,3.940,4.190,4.250,1.375,3.310,0.500,3.690
Acupuncturists,3.310,3.685,3.125,2.940,3.815,0.50,0.875,0.500,2.685,3.500,...,2.370,3.500,3.685,3.560,2.685,2.935,1.310,2.685,1.125,3.375
Acute Care Nurses,3.815,3.940,3.750,3.815,4.000,0.50,1.440,0.500,3.690,3.875,...,3.060,3.940,4.310,4.000,3.060,3.125,1.560,3.185,1.810,3.560
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Wind Energy Project Managers,3.690,3.750,3.315,3.750,4.125,0.75,0.815,0.500,3.065,3.560,...,1.435,3.000,3.560,3.815,3.060,3.190,1.375,3.500,1.690,3.880
Wind Turbine Service Technicians,2.940,3.065,3.190,3.250,3.310,3.56,2.750,1.500,3.000,3.060,...,2.060,2.315,2.625,3.000,2.815,2.815,1.940,2.815,3.750,2.620
"Woodworking Machine Setters, Operators, and Tenders, Except Sawing",2.310,2.815,2.625,2.060,2.940,3.00,2.690,1.565,1.815,2.500,...,1.000,1.630,2.125,2.750,1.315,1.810,1.630,2.435,3.000,1.815


# K-Means Cluster 

In [7]:
kmeans = KMeans(n_clusters = 7,random_state = 5)
kmeans


KMeans(n_clusters=7, random_state=5)

In [8]:
Kmodel = kmeans.fit(df_skills)
# Kmodel.labels_

In [9]:
# Kmodel.cluster_centers_

In [10]:
def clusters(df,cluster_list):
    '''
    @param: df: dataframe of occupations
            cluster_list: list of different clusters
    returns: dictionary of cluster and their respective occupations 
    '''
    occ_dict = {}
    for i in range(7):
        occ_dict[i] = set()
    
    for occ,cl in zip(df.reset_index()['title'],cluster_list):
        occ_dict[cl].add(occ)
    return occ_dict  

In [11]:
df = reduce(lambda x,y: pd.merge(x,y, on='title', how='outer'), [df_wc,df_knowledge,df_skills,df_gwas,df_jz])
df = df.set_index('title')
df[:20]

,Consequence of Error,Contact With Others,Coordinate or Lead Others,"Cramped Work Space, Awkward Positions",Deal With External Customers,Deal With Physically Aggressive People,Deal With Unpleasant or Angry People,Degree of Automation,Electronic Mail,Exposed to Contaminants,...,Repairing and Maintaining Electronic Equipment,Repairing and Maintaining Mechanical Equipment,Resolving Conflicts and Negotiating with Others,Scheduling Work and Activities,Selling or Influencing Others,Staffing Organizational Units,Thinking Creatively,Training and Teaching Others,Updating and Using Relevant Knowledge,job_zone
title,,,,,,,,,,,,,,,,,,,,,
Accountants,2.88,4.59,3.05,1.12,3.92,1.03,2.25,3.20,4.97,1.23,...,0.810,0.830,3.285,3.270,2.335,2.210,3.360,3.560,3.795,4.0
Actors,1.97,4.71,3.16,1.89,3.65,1.48,2.67,1.35,4.68,2.29,...,0.850,0.865,2.130,2.275,1.720,1.005,4.525,2.490,2.050,2.0
Actuaries,3.28,3.62,3.12,1.06,2.66,1.03,1.91,2.50,5.00,1.00,...,0.590,0.500,3.060,3.390,2.890,3.080,4.330,3.475,4.940,4.0
Acupuncturists,2.90,4.80,3.18,1.52,3.70,1.36,2.51,1.44,4.29,1.78,...,1.225,1.040,2.725,2.390,2.690,1.180,4.095,1.975,3.795,5.0
Acute Care Nurses,4.77,4.81,4.00,2.89,4.52,3.00,4.07,2.46,4.48,3.56,...,1.535,1.225,4.425,3.800,2.445,3.325,3.500,4.075,4.630,3.0
Adapted Physical Education Specialists,2.47,4.31,3.67,2.13,2.97,3.17,2.87,1.32,4.62,2.35,...,1.160,1.725,3.830,3.705,2.560,1.750,4.795,4.580,4.605,4.0
Adhesive Bonding Machine Operators and Tenders,3.82,4.61,3.27,2.92,2.32,2.08,2.99,2.84,2.58,4.78,...,2.390,3.315,1.595,2.400,1.385,1.255,2.200,3.225,2.920,2.0
"Administrative Law Judges, Adjudicators, and Hearing Officers",3.20,4.60,3.74,1.17,4.46,2.53,4.22,2.38,4.67,1.87,...,0.655,0.600,4.395,3.505,2.580,1.515,3.685,2.685,4.755,5.0
Administrative Services Managers,2.77,4.50,3.90,1.52,4.12,1.75,3.08,2.14,4.96,2.12,...,2.415,2.500,4.190,4.640,2.370,3.505,3.855,3.510,4.430,3.0


In [18]:
kmeans = KMeans(n_clusters = 7,random_state = 5)
Kmodel = kmeans.fit(df[:50])

clusters(df[:50],Kmodel.labels_)

{0: {'Acupuncturists',
  'Acute Care Nurses',
  'Advanced Practice Psychiatric Nurses',
  'Allergists and Immunologists',
  'Anesthesiologist Assistants',
  'Anesthesiologists'},
 1: {'Adhesive Bonding Machine Operators and Tenders',
  'Aerospace Engineering and Operations Technicians',
  'Agricultural Equipment Operators',
  'Aircraft Mechanics and Service Technicians',
  'Aircraft Structure, Surfaces, Rigging, and Systems Assemblers',
  'Airline Pilots, Copilots, and Flight Engineers'},
 2: {'Accountants',
  'Actuaries',
  'Administrative Law Judges, Adjudicators, and Hearing Officers',
  'Administrative Services Managers',
  'Advertising Sales Agents',
  'Advertising and Promotions Managers',
  'Agents and Business Managers of Artists, Performers, and Athletes',
  'Appraisers, Real Estate',
  'Architectural Drafters',
  'Archivists',
  'Art Directors'},
 3: {'Aerospace Engineers',
  'Agricultural Engineers',
  'Agricultural Sciences Teachers, Postsecondary',
  'Animal Scientists',
 

In [14]:
# Next steps: How many clusters? What factors to take into account of?


In [15]:
# Mean-Shift

In [16]:
# DBSCAN...not take into account of "noise" (i.e. not so relevant occupations?) 